## New Objective Development in Adaptor

This tutorial will show you how to develop a new objective in Adaptor environment.

#### Why would I develop a new objective?

In Adaptor, you can use Objectives to **train** and/or **evaluate** your model on a selected, separate set of data.

There are also successful applications of multi-objective training, where a composition of more than one objectives increase measured quality of the final model. It is much easier to do the multi-objective training in the objective-centric manner.

Note that if you just want to evaluate your model, you should suffice with a new [Evaluator](https://github.com/gaussalgo/adaptor/blob/master/adaptor/evaluators/evaluator_base.py#L10) instance. Refer to a section **Custom Evaluators** in [adapted_named_entity_recognition](https://github.com/gaussalgo/adaptor/blob/master/tutorials/adapted_named_entity_recognition.ipynb) tutorial.

## Implementation

Normally, you would perhaps implement this in your experimental code base, but for a transparency, we'll add a full implementation here.

### Standard process

Adaptor's Objectives need to choose one attribute and implement two methods, pre-defined in [`adaptor.objectives.objective_base.Objective`](https://github.com/gaussalgo/adaptor/blob/5888436b6cd1fa73805767e16754d37a514fcf36/adaptor/objectives/objective_base.py#L18) abstract class: 

1. Choose `compatible_head: Head` that will define what type of the head your objective expects. If your use-case does not fit either of the commonly-used transformer heads, you can omit this and fill in `objective_module: torch.nn.Module` attribute of the `ObjectiveInstance.__init__()`.
1. Implement or inherit `_get_inputs_iterator(self, split: str) -> Iterable[Union[BatchEncoding, Dict[str, torch.Tensor]]]:`: the method that will provide objective-spefic **input encodings** to the selected Objective's head.
2. Implement or inherit `_compute_loss(self, inputs, logit_outputs, labels) -> torch.FloatTensor:` that will compute a scalar of a **loss** with grad_fn.

...

Usually, you proceed with this checklist:

1. **Identify custom parts.** How does your objective differ from the existing ones? This defines the functionality that you will likely need to implement.
2. **Choose the ancestors.** You start with choosing the best-suitable abstract class for your new Objective, that might already implement a part of the functionality above for you. It's also a good idea to take a look at **similar objectives** and choose one or more to inherit from, or perhaps just to take over some of their functionality (e.g. batched inputs encoding).
3. **Test it.** Even in a standalone, it's quite easy to test a new Objective: You can just **retrieve the sample** from the new objective, and **compute the loss** on it. You can find such tests over all Adaptor's Objectives in [objectives_test](https://github.com/gaussalgo/adaptor/blob/5888436b6cd1fa73805767e16754d37a514fcf36/tests/objectives_test.py#L42) or also below with our new objective.

If this test passes, you are good to use the new objective in the training process.

### Problem

Say that we want to create an objective that will sanction a usage of a specific word in translation.

Since using this one objective in a standalone will probably do the job, but might disrupt the original quality of the adapted translator, we'll use this objective in a composition with a standard Sequence2Sequence objective (i.e. MLE objective).

In [63]:
from adaptor.objectives.seq2seq import Sequence2SequenceMixin
from adaptor.objectives.objective_base import UnsupervisedObjective

import torch

from transformers import BatchEncoding
from typing import Optional, Dict, Union


class TokenSanctioningObjective(Sequence2SequenceMixin, UnsupervisedObjective):
    
    def __init__(self, sanctioned_token_id: int, *args, **kwargs):
        
        self.sanctioned_token_id = sanctioned_token_id
        super().__init__(*args, **kwargs)
        
    
    def _compute_loss(self,
                      logit_outputs: Optional[torch.FloatTensor] = None,
                      labels: Optional[torch.LongTensor] = None,
                      inputs: Optional[Union[BatchEncoding, Dict[str, torch.Tensor]]] = None) -> torch.FloatTensor:
        """
            Computes loss on the separately-selected, sancioned token.
            :param logit_outputs: Raw outputs of language modeling head model
            :param labels: Token ids of expected outputs. Not used here; expected probabilities of the sanctioned token are zero.
            :param inputs: Model inputs. Not used here.

            :return: A scalar of the loss with grad_fn.
        """
        # note that currently we do not ignore padding from the loss, which might be desirable
        # - we have seen this to eliminate repetitive generations at some cases
        loss_fct = torch.nn.CrossEntropyLoss()
        
        sanctioned_logits = logit_outputs[..., self.sanctioned_token_id]
        sanctioned_token_loss = loss_fct(sanctioned_logits, torch.zeros_like(sanctioned_logits))

        return sanctioned_token_loss

### Testing a New Objective

In [64]:
from adaptor.lang_module import LangModule

lang_module = LangModule("facebook/bart-base")

objective_inst = TokenSanctioningObjective(sanctioned_token_id=123,
                                           lang_module=lang_module,
                                           texts_or_path=["A piece of text", "Another piece of text"],
                                           batch_size=1)

# you can sample either directly from `_get_inputs_iterator` or from the wrapping top-level `get_dataset`
dataset_sample = next(iter(objective_inst.get_dataset(split="train", objective_i=0, device="cpu")))
dataset_sample

TokenSanctioningObjective:   0%|          | 0/2 [00:00<?, ?batches/s]

{'input_ids': tensor([[   0,  250, 2125,    9, 2788,    2,    1,    1]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0]]),
 'labels': tensor([[   0,  250, 2125,    9, 2788,    2]]),
 'decoder_input_ids': tensor([[   2,    0,  250, 2125,    9, 2788]]),
 'oid': 139875940821984}

In [65]:
outputs = lang_module(**dataset_sample)
outputs

tensor([[[33.9664,  6.6620, 16.1622,  ...,  6.4252,  6.3618,  1.4679],
         [ 6.8863, -2.6064, 14.3440,  ..., -3.1052, -2.9539, -3.0129],
         [-8.1249, -5.2164,  4.4455,  ..., -5.1076, -5.3682, -3.9794],
         [ 0.9497, -4.5346, 10.3620,  ..., -4.9138, -5.4243,  0.3182],
         [-6.0416, -5.6131,  5.0793,  ..., -6.5252, -6.5287, -4.9718],
         [-0.3254, -3.9097, 18.0728,  ..., -4.3769, -4.0694, -1.8632]]],
       grad_fn=<AddBackward0>)

In [ ]:
loss = objective_inst.compute_loss(logit_outputs=outputs, labels=None, split="train")

In [69]:
loss.backward()

### Training

From here on, we use the new objective on the data of our desire like any other objective.

Since we still want to model to learn to translate text, we perhaps want to combine the sancioning objective with a the traditional Sequence2Sequence and train on the both objectives in parallel.

In [ ]:
lang_module = LangModule("Helsinki-NLP/opus-mt-en-cs")

In [ ]:
seq_objective = Sequence2Sequence(lang_module,
                                  texts_or_path=["A piece of text", "Another piece of text"],
                                  labels_or_path=["Kus textu", "Další kus textu],
                                  source_lang_id="en",
                                  target_lang_id="cs",
                                  batch_size=1,
                                  objective_id="Opensub")

In [ ]:
sanctioning_objective = TokenSanctioningObjective(sanctioned_token_id=123,
                                                  lang_module=lang_module,
                                                  texts_or_path=["A piece of text", "Another piece of text"],
                                                  batch_size=1,
                                                  share_other_objective_head=seq_objective)

In [ ]:
from adaptor.schedules import ParallelSchedule

schedule = ParallelSchedule(objectives=[seq_objective, sanctioning_objective])

In [ ]:
from adaptor.adapter import Adapter

adapter = Adapter(lang_module, schedule, args=training_arguments)
adapter.train()

In [ ]:
adapter.save_model("final_translation_model")

See [unsupervised_machine_translation](https://github.com/gaussalgo/adaptor/blob/master/tutorials/unsupervised_machine_translation.ipynb)
tutorial for a running example with mock data and further analysis!